In [1]:
global sequence, P, Q, L, R, M, Pi, Qi, Li, Ri, IS1, IS2, min_loop
global W, V, Wi, TB
global w_progress, v_progress, wi_progress

In [2]:
def zuker(seq, P_funct, Q_const, L_funct, R_funct, M_const, Pi_funct, Qi_funct, Li_funct, Ri_funct, IS1_funct, IS2_funct, min_loop_const) :
    global sequence, P, Q, L, R, M, Pi, Qi, Li, Ri, IS1, IS2, min_loop, w_progress, v_progress, wi_progress
    sequence, P, Q, L, R, M, Pi, Qi, Li, Ri, IS1, IS2, min_loop = seq, P_funct, Q_const, L_funct, R_funct, M_const, Pi_funct, Qi_funct, Li_funct, Ri_funct, IS1_funct, IS2_funct, min_loop_const
    global W, V, Wi, TB
    W, V, Wi, TB = [], [], [], []
    w_progress, v_progress, wi_progress = 0, 0, 0
    init_matrices()
    wx(0,len(sequence) - 1)
    return traceback()


In [3]:
def init_matrices():
    global W, V, Wi, TB
    global w_progress, v_progress, wi_progress
    global sequence
    for x in range(len(sequence)):
        V.append([])
        W.append([])
        Wi.append([])
        TB.append([])
        for y in range(len(sequence)):
            V[x].append('-')
            W[x].append('-')
            Wi[x].append('-')
            TB[x].append('-')
    for i in reversed(range(len(V))):
        for j in range(len(V[i])):
            if j - i <= m:
                V[i][j] = float('inf')
                W[i][j] = 0
                Wi[i][j] = 0
                TB[i][j] = 6
                w_progress += 1
                v_progress += 1
                wi_progress += 1


In [4]:
def w_print(W):
    for char in sequence:
        print(char + "     ", end = "")
    print()
    for m in range(len(W)):
        for n in range(len(W[m])):
            string = str(W[m][n])
            for x in range(len(string), 6) :
                string = string + " "
            print(string, end = "")
        print(sequence[m])

In [5]:
def wx(i,j):
    if(j - i < m) :
        return 0
    if W[i][j] != "-":
        return W[i][j]
    min_k = get_min_k(i, j)
    possibilities = [P(i,j) + vx(i,j), L(i, i+1, j-1) + R(j, i+1, j-1) + P(i + 1,j - 1) + vx(i+1, j-1),
                     L(i, i+1, j) + P(i + 1,j) + vx(i+1, j), R(j, i, j-1) + P(i,j - 1) + vx(i, j-1),
                    Q + wx(i + 1, j), Q + wx(i, j - 1), min_k[0]]
    W[i][j] = min(possibilities)
    TB[i][j] = possibilities.index(W[i][j])
    if TB[i][j]  == 6:
        TB[i][j] = min_k[1] * -1
    W[i][j] = round(W[i][j], 1)
    global w_progress, v_progress, wi_progress
    w_progress += 1
    print('W: {}/{}; V: {}/{}; Wi: {}/{}\t{}%'.format(w_progress, len(W)*len(W), v_progress, len(W)*len(W), wi_progress, len(W)*len(W), round(w_progress/ (len(W)*len(W)) * 100, 3)), end='\r')

    return W[i][j]
def get_min_k(i, j):
    minimum = float('inf')
    min_k = -1
    for k in range (i + 1, j - 1) :
        result = wx(i,k) + wx(k + 1,j)
        if minimum > result :
            minimum = result
            min_k = k
    return minimum, min_k

In [6]:
def vx(i,j):
    global P
    if(P(i,j) == float('inf')):
        return float('inf')
    if(j - i < m) :
        return float('inf')
    if V[i][j] != "-":
        return V[i][j]
    possibilities = [IS1(i,j), optimal_IS2(i,j), optimal_IS3(i,j)]
    global w_progress, v_progress, wi_progress
    v_progress += 1
    print('W: {}/{}; V: {}/{}; Wi: {}/{}\t{}%'.format(w_progress, len(W)*len(W), v_progress, len(W)*len(W), wi_progress, len(W)*len(W), round(w_progress/ (len(W)*len(W)) * 100, 3)), end='\r')
    V[i][j] = round(min(possibilities), 2)
    return min(possibilities)
    

In [7]:
def optimal_IS3(i, j) :
    minimum = float('inf')
    for k in range (i + 1, j - 1) :
        if k - i - 1 >= m and j - k - 1 >= m:
            multi = M + wxi(i + 1,k) + wxi(k + 1,j - 1)
            if minimum > multi :
                minimum = multi
    return minimum

In [8]:
def optimal_IS2(i, j):
    global P
    if(P(i,j) == float('inf')):
        return float('inf')
    minimum = float('inf')
    for k in range(i + 1, j - 1):
        for l in range (k + 1, j):
            is2 = IS2(i, j, k, l) + P(k,l) + vx(k, l)
            if(is2 < minimum):
                minimum = is2
    return minimum

In [9]:
def wxi(i, j):
    if(j - i < m) :
        return 0
    if Wi[i][j] != "-":
        return Wi[i][j]
    min_k = get_min_k(i, j)
    possibilities = [Pi(i,j) + vx(i,j), Li(i, i+1, j-1) + Ri(j, i+1, j-1) + Pi(i + 1,j - 1) + vx(i+1,j-1),
                     Li(i, i+1, j) + Pi(i + 1,j) + vx(i+1,j), Ri(j, i, j-1) + Pi(i,j - 1) + vx(i,j-1),
                    Qi + wxi(i + 1,j), Qi + wxi(i,j - 1), min_k[0]]
    Wi[i][j] = min(possibilities)
    TB[i][j] = possibilities.index(Wi[i][j])
    if(TB[i][j]) == 6:
        TB[i][j] = min_k[1] * -1
    Wi[i][j] = round(Wi[i][j], 1)
    return min(possibilities);

In [10]:
global structure

def traceback():
    global structure
    structure = []
    for i in range(len(sequence)):
        structure.append("•")
    i = 0
    j = len(sequence) - 1
    traceback_recurse(i, j)
    string = ""
    for i in range(len(structure)):
        string +=(structure[i])
    return string
def traceback_recurse(i, j):
    global structure
    if TB[i][j] == 0:
        structure[i] = '('
        structure[j] = ')'
        traceback_recurse(i+1, j-1)
    elif TB[i][j] == 1:
        traceback_recurse(i+1, j-1) 
    elif TB[i][j] == 2 or TB[i][j] == 4:
        traceback_recurse(i+1, j)
    elif TB[i][j] == 3 or TB[i][j] == 5:
        traceback_recurse(i, j-1)
    elif TB[i][j] < 0:
        traceback_recurse(i, TB[i][j] * -1)
        traceback_recurse(TB[i][j] * -1 + 1, j)
    

In [13]:
Q = 0
M = 4.6
Qi = 0
m = 3
sequence = "GGUGGCAGCGGCGGUUGAAUUGCCAUGGCAAUGCGGUGGGCGCGCGCUUGUCGUGUUGGUCUCUUGGGAGGUAGUGGGGCUAGGCCGGGCGGGUAUCCGCCUCUCCCAGCUUAGGUGAGCGUCCCCGGGCGCCUCCGGAGCGCCGCGGCCGCAUGCAGUUCGUCGUGGCGGGGAGCCGGAGCCUGACCGGGGUUCCAGCGCUCGGGCCGUAGCCUUGGCUCCUGGACUUUCCCUGGCUCCGCCGCCACGUGGGAGCUGAGGCUCUGGGGCUUCCGCCUCCGGCGCGCGAUUAUUUCUCUAGAACAGUUUUCAUUUUUAAAAUUUGUAAAGCGCUUUUGCCUGUGUGAUUUCCUCUGGGUUUUUUUUUUUUUUUCUUCCUUUUUGUAGAGACGGAAUUGGCGGCGGGGGCGGGGGGUCGAUGUCUCACUUUUUUGCCCAGGCUGGUCUCGAACUCCUGGCUUCAAGGGAUCCUCAUGCCUCGGCCUCUUAAAGUGCUGGGAUUACAGGCGUGAGCCACGCCCCCGGCCGCCUCUGAGUUUCCAGCCUCGUUGGCCCUCCAGCCUUUUAACCUGUUGGGCCUAGGAUCAGGAAAGGUUUGUUGAAUGGGGAACUAAGAAGUGAAUUCGUUCGUUCGACAAACGUUUCCUGAGCAGCCGCUGGGUGCUAGGCGCAGUGCCAGCGCGGAAUGUCCAGGGAGACCUGGUGCCCAAAGCUUGGACCCAUCGUGAGAAAUGAGAAGCAGAUACAAAGCAGUGUGGGAGUGCAGAGGAGACAAAGCAAGCCUCAUCAGGCCCAUUGCUUGCUCUGCUCUCCCUUGUACUUACCAGUGCUUGACAAUAUACAGUUAUUUACUAGCUUGGUUAUUGACUUCCUAUCCAGCACUCAGUUUUAUUCACUGCUGUAUCCUCAGUGCCUAGGACGAUGCUUGGAACGUGGUAAGUGCUCCUAUUGGCGGGAAGAAUAAAUCCGGAAGAGCAGGACCAGUGGACUUGCUACAUAA"

# seq, P_funct, Q_const, L_funct, R_funct, M_const, Pi_funct, Qi_funct, Li_funct, Ri_funct, IS1_funct, IS2_funct, min_loop_const
global W, V

print(sequence)
print(zuker(sequence, FreierPair, Q, FreierDangleLeft, FreierDangleRight, M, FreierPair, Qi, FreierDangleLeft, FreierDangleRight, FreierIS1, FreierIS2, m))


GGUGGCAGCGGCGGUUGAAUUGCCAUGGCAAUGCGGUGGGCGCGCGCUUGUCGUGUUGGUCUCUUGGGAGGUAGUGGGGCUAGGCCGGGCGGGUAUCCGCCUCUCCCAGCUUAGGUGAGCGUCCCCGGGCGCCUCCGGAGCGCCGCGGCCGCAUGCAGUUCGUCGUGGCGGGGAGCCGGAGCCUGACCGGGGUUCCAGCGCUCGGGCCGUAGCCUUGGCUCCUGGACUUUCCCUGGCUCCGCCGCCACGUGGGAGCUGAGGCUCUGGGGCUUCCGCCUCCGGCGCGCGAUUAUUUCUCUAGAACAGUUUUCAUUUUUAAAAUUUGUAAAGCGCUUUUGCCUGUGUGAUUUCCUCUGGGUUUUUUUUUUUUUUUCUUCCUUUUUGUAGAGACGGAAUUGGCGGCGGGGGCGGGGGGUCGAUGUCUCACUUUUUUGCCCAGGCUGGUCUCGAACUCCUGGCUUCAAGGGAUCCUCAUGCCUCGGCCUCUUAAAGUGCUGGGAUUACAGGCGUGAGCCACGCCCCCGGCCGCCUCUGAGUUUCCAGCCUCGUUGGCCCUCCAGCCUUUUAACCUGUUGGGCCUAGGAUCAGGAAAGGUUUGUUGAAUGGGGAACUAAGAAGUGAAUUCGUUCGUUCGACAAACGUUUCCUGAGCAGCCGCUGGGUGCUAGGCGCAGUGCCAGCGCGGAAUGUCCAGGGAGACCUGGUGCCCAAAGCUUGGACCCAUCGUGAGAAAUGAGAAGCAGAUACAAAGCAGUGUGGGAGUGCAGAGGAGACAAAGCAAGCCUCAUCAGGCCCAUUGCUUGCUCUGCUCUCCCUUGUACUUACCAGUGCUUGACAAUAUACAGUUAUUUACUAGCUUGGUUAUUGACUUCCUAUCCAGCACUCAGUUUUAUUCACUGCUGUAUCCUCAGUGCCUAGGACGAUGCUUGGAACGUGGUAAGUGCUCCUAUUGGCGGGAAGAAUAAAUCCGGAAGAGCAGGACCAGUGGACUUGCUACAUAA

KeyboardInterrupt: 

In [ ]:
w_print(W)
w_print(V)
w_print(TB)

In [14]:
sequence = "GCGACUCGGGGUGCCCUCCAUUGCACUCCGGAGGCUGAGAAAUACCCGUAUCACCUGAUCUGGAUAAUGCCAGCGUAGGGAAGUCGC"
print(sequence)
print(zuker(sequence, FreierPair, Q, FreierDangleLeft, FreierDangleRight, M, FreierPair, Qi, FreierDangleLeft, FreierDangleRight, FreierIS1, FreierIS2, m))

GCGACUCGGGGUGCCCUCCAUUGCACUCCGGAGGCUGAGAAAUACCCGUAUCACCUGAUCUGGAUAAUGCCAGCGUAGGGAAGUCGC


W: 5353/7569; V: 4405/7569; Wi: 5232/7569	70.723%

(((•(•((((•••)))•((••(•(•••••)•))((•(•(•••••)•))•(((••••)))((((••••••)))))•••)•)••)•)))


In [32]:
w_print(W)
w_print(V)
w_print(TB)

G     G     U     G     G     C     A     G     C     G     G     C     G     G     U     U     G     A     A     U     U     G     C     C     A     U     G     G     C     A     A     U     G     C     G     G     U     G     G     G     C     G     C     G     C     G     C     U     U     G     U     C     G     U     G     U     U     G     G     U     C     U     C     U     U     G     G     G     A     G     G     U     A     G     U     G     G     G     G     C     U     A     G     G     C     C     G     G     G     C     G     G     G     U     A     U     C     C     G     C     C     U     C     U     C     C     C     A     G     C     U     U     A     G     G     U     G     A     G     C     G     U     C     C     C     C     G     G     G     C     G     C     C     U     C     C     G     G     A     G     C     G     C     C     G     C     G     G     C     C     G     C     A     U     G     C     A     G     U     U     C     G     U     C     G     U     G   

0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

0     0     0     0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

0     0     0     0     0     0     0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

0     0     0     0     0     0     0     0     0     0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

...

0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -     -   

KeyboardInterrupt: 

In [12]:
import numpy

def FreierDangleLeft(x, a, b):
    five_prime = {"A":{"A":-0.3, "C":-0.5, "G":-0.2,"U":-0.3},
                  "C":{"A":-0.3, "C":-0.2, "G":-0.3,"U":-0.2},
                  "G":{"A":-0.4, "C":-0.2, "G":-0.0,"U":-0.2},
                  "U":{"A":-0.2, "C":-0.1, "G":-0.0,"U":-0.2}}
    return five_prime[sequence[x]][sequence[a]]

def FreierDangleRight(x, a, b):
    three_prime = {"A":{"A":-0.8, "C":-1.7, "G":-1.1,"U":-0.7},
                  "C":{"A":-0.5, "C":-0.8, "G":-0.4,"U":-0.1},
                  "G":{"A":-0.8, "C":-1.7, "G":-1.3,"U":-0.7},
                  "U":{"A":-0.6, "C":-1.2, "G":-0.6,"U":-0.1}}
    return three_prime[sequence[x]][sequence[a]]

def FreierStack(i, j):
    if sequence[i] == 'G' and sequence[j] == 'C':
        scores = {"A":{"A":-1.1, "C":-1.1, "G":-1.6,"U":-2.1},
                  "C":{"A":-1.3, "C":-0.6, "G":-3.4,"U":-0.5},
                  "G":{"A":-1.3, "C":-3.4, "G":-1.4,"U":-2.3},
                  "U":{"A":-2.3, "C":-0.5, "G":-1.4,"U":-0.7}}
        return scores[sequence[i + 1]][sequence[j - 1]]
    elif sequence[i] == 'C' and sequence[j] == 'G':
        scores = {"A":{"A":-1.9, "C":-1.0, "G":-1.9,"U":-1.7},
                  "C":{"A":-2.0, "C":-1.1, "G":-2.0,"U":-1.5},
                  "G":{"A":-1.9, "C":-2.9, "G":-1.9,"U":-1.9},
                  "U":{"A":-1.8, "C":-0.8, "G":-1.6,"U":-1.2}}
        return scores[sequence[i + 1]][sequence[j - 1]]
    elif sequence[i] == 'A' and sequence[j] == 'U':
        scores = {"A":{"A":-0.8, "C":-0.7, "G":-0.8,"U":-0.9},
                  "C":{"A":-1.0, "C":-0.7, "G":-1.7,"U":-0.8},
                  "G":{"A":-1.0, "C":-2.1, "G":-1.0,"U":-0.9},
                  "U":{"A":-0.9, "C":-0.7, "G":-0.9,"U":-0.8}}
        return scores[sequence[i + 1]][sequence[j - 1]]
    elif sequence[i] == 'U' and sequence[j] == 'A':
        scores = {"A":{"A":-1.0, "C":-0.7, "G":-1.1,"U":-0.9},
                  "C":{"A":-0.8, "C":-0.6, "G":-1.8,"U":-0.6},
                  "G":{"A":-1.1, "C":-1.8, "G":-1.2,"U":-1.0},
                  "U":{"A":-1.1, "C":-0.5, "G":-0.9,"U":-0.5}}
        return scores[sequence[i + 1]][sequence[j - 1]]
    return 0

def FreierIS1(i,j):
    length = j - i - 1
    if length < m :
        return float('inf')
    values = {3 : 7.4, 4 : 5.9, 5 : 4.4, 6 : 4.3, 7 : 4.1, 8 : 4.1, 9 : 4.2, 10 : 4.3, 12 : 4.9, 14 : 5.6, 16 : 6.1, 18 : 6.7, 20: 7.1, 25 : 8.1, 30 : 8.9}
    if length in values :
        return values[length]
    else :
        return round(4.0476 * numpy.log(length) - 4.9884, 1)

def FreierIS2(i, j, k, l) :
    is2= float('inf')
    if k - 1 == i and l + 1 == j:
        is2 = FreierStack(k, l)
    if k - 1 == i and l + 1 < j or k - 1 > i and l + 1 == j:
        is2 = FreierBulge(l - k - 1)
    else:
        is2 = FreierInternalLoop(l - k - 1)
    return is2

def FreierBulge(length):
    if length < 1:
        return float('inf')
    values = {1 : 3.3, 2 : 5.2, 3 : 6.0, 4 : 6.7, 5 : 7.4, 6 : 8.2, 7 : 9.1, 8 : 10.0, 9 : 3.1, 10 : 3.6, 12 : 4.4, 14 : 5.1, 16 : 5.6, 18 : 6.2, 20: 6.6, 25 : 7.6, 30 : 8.4}
    if length in values :
        return values[length]
    else :
        return round(4.3834 * numpy.log(length) + 0.8927, 1)

def FreierInternalLoop(length):
    if length < 2 :
        return float('inf')
    values = {2 : 0.8, 3 : 1.3, 4 : 1.7, 5 : 2.1, 6 : 2.5, 7 : 2.6, 8 : 2.8, 9 : 3.1, 10 : 3.6, 12 : 4.4, 14 : 5.1, 16 : 5.6, 18 : 6.2, 20: 6.6, 25 : 7.6, 30 : 8.4}
    if (length) in values :
        return values[length]
    else :
        return round(pow(length,2)*-0.0045 + 0.4185*(length)- 0.0003, 1)

def FreierPair(i, j):
    p = 0
    if sequence[i] == 'A' and sequence[j] == 'U' or sequence[i] == 'U' and sequence[j] == 'A' :
        p = -4.15
    elif sequence[i] == 'G' and sequence[j] == 'C' or sequence[i] == 'C' and sequence[j] == 'G' :
        p = -6.17
    else :
        p = float('inf')
    return p + FreierStack(i, j)